In [171]:
import sklearn
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import copy #For deepcopy
import miceforest
from miceforest import ImputationKernel #Multivariate imputation technique for missing values

from matplotlib import rcParams

from sklearn.preprocessing import StandardScaler

from scipy import stats
from sklearn.model_selection import train_test_split

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import seaborn as sns 
from sklearn import svm

In [172]:
df = pd.read_csv('train.csv') #Import
PassengerList = df.PassengerId.values #Save passenger id

In [173]:
### Feature enginnering featureEng()
#Split Columns
df[["temp", "Number"]] = df["PassengerId"].str.split("_", expand=True)
df['Number'] = df['Number'].astype('Int64') #'n' -> n : n ∈ N
df[["Deck", "CabinNum", "Side"]] = df["Cabin"].str.split("/", expand=True)
#Drop columns
df = df.drop(columns=["PassengerId"])
df = df.drop(columns=["temp"])
df = df.drop(columns=["Cabin"])
df = df.drop(columns=["Name"])

### Group values
#Cabin number is converted into categoriacal values
df['CabinNum'] = df['CabinNum'].astype('Int64') #'n' -> n : n ∈ N
df['CabinNum'] = pd.cut(df['CabinNum'],bins=[0,250,500,750,1000,1250,1500],labels=['A', 'B', 'C', 'D', 'E', 'F'])

#Age is converted into categoriacal values.
df['Age'] = pd.cut(df['Age'],bins=[0,2,17, 35,65,99],labels=['Toddler','Child','Young adult','Adult','Elderly'])
df['Spendings'] = df['RoomService'] + df['FoodCourt'] + df['ShoppingMall'] + df['Spa'] + df['VRDeck']

### convert categorical variable into Numeric numConverter()
df['HomePlanet'] = pd.factorize(df['HomePlanet'])[0]
df['CryoSleep'] = pd.factorize(df['CryoSleep'])[0]
df['Destination'] = pd.factorize(df['Destination'])[0]
df['Age'] = pd.factorize(df['Age'])[0]
df['VIP'] = pd.factorize(df['VIP'])[0]
df['Deck'] = pd.factorize(df['Deck'])[0]
df['CabinNum'] = pd.factorize(df['CabinNum'])[0]
df['Side'] = pd.factorize(df['Side'])[0]
df = df.replace(-1, np.NaN)
df

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Number,Deck,CabinNum,Side,Spendings
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,9.0,0.0,2823.0,0.0,1,1.0,0.0,0.0,2832.0
2,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,2.0,NaN,0.0,0.0
3,1.0,1.0,0.0,1.0,0.0,0.0,6652.0,0.0,181.0,585.0,1,2.0,0.0,0.0,7418.0
4,0.0,1.0,0.0,0.0,0.0,10.0,0.0,635.0,0.0,0.0,1,1.0,0.0,0.0,645.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,0.0,5.0,0.0,0.0
4273,0.0,1.0,0.0,1.0,0.0,0.0,847.0,17.0,10.0,144.0,1,NaN,NaN,NaN,1018.0
4274,2.0,0.0,1.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1,4.0,1.0,1.0,0.0
4275,1.0,1.0,NaN,NaN,0.0,0.0,2680.0,0.0,0.0,523.0,1,4.0,1.0,1.0,3203.0


In [174]:
df.to_csv('df_1.csv', index = False) 

In [175]:
mice_kernel = ImputationKernel(
    data = df,
    save_all_iterations = True,
    random_state = 42)
mice_kernel.mice(2)
df_ = mice_kernel.complete_data()
for elm in df_: df[elm] = df_[elm]

In [176]:
df.to_csv('df_2.csv', index = False) 